In [2]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS



newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
vectorizer = CountVectorizer(lowercase=True, stop_words=ENGLISH_STOP_WORDS,analyzer='word', binary=True)
X_train = vectorizer.fit_transform(newsgroups_train.data)

X_train.toarray()
# X_train.shape #(11314, 101322)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [3]:
from tqdm import tqdm
def lda(n_dk,n_kw,n_k,z,docs,words,alpha,beta,NITER):
    for tek_iter in tqdm(range(NITER)):
        for i in range(N):
            n_dk[docs[i],z[i]]-=1
            n_kw[z[i],words[i]]-=1
            n_k[z[i]]-=1
            p = (n_dk[docs[i], :] + alpha) * (n_kw[:, words[i]] + beta[words[i]]) / (n_k + beta.sum())
            z[i] = np.random.choice(np.arange(K), p=p / p.sum())
            
        
            n_dk[docs[i],z[i]]+=1
            n_kw[z[i],words[i]]+=1
            n_k[z[i]]+=1
    return  n_dk, n_kw,n_k,z

In [5]:
import numpy as np
M=11314 #amount of texts
W=101322 #amount of different words
N=755809#total amount of words in the corpus
K=20 #amount of tags
NITER=10


alpha=2*np.ones(K)
beta=2*np.ones(N)

n_dk=np.zeros(M*K).reshape(M,K) #amount of words in document d assigned to tag k
n_kw=np.zeros(K*W).reshape(K,W)
n_k=np.zeros(K) #total amount of words assigned to tag k

X=X_train.toarray()
docs,words=X.nonzero() #print(len(docs)) #it is N=755809
z=[np.random.choice(K) for i in range(N)] #randomly assign tags to words
for doc, word, cur_z in zip(docs, words, z):
        n_dk[doc, cur_z] += 1
        n_kw[cur_z, word] += 1
        n_k[cur_z] += 1
print(alpha[0:10])

[2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]


In [6]:
n_dk,n_kw,n_k,z=lda(n_dk,n_kw,n_k,z,docs,words,alpha,beta,NITER)
#print(n_kw)

100%|██████████| 10/10 [58:12<00:00, 348.84s/it]


In [10]:
n_kw_sorted=np.argsort(n_kw, axis=1)
#print(vectorizer.vocabulary_.get('car'))
#print(type(vectorizer.vocabulary_))

my_dict={}
for item in vectorizer.vocabulary_:
    my_dict[vectorizer.vocabulary_[item]]=item

for k in range(K):
    print("tag=",k,end=" ")
    for j in range(10):
        print(my_dict[n_kw_sorted[k,W-1-j]],end=" ")
    print("\n")

tag= 0 wing dk immaculate 8v bxn ns sells xi hm rusty 

tag= 1 wt 3t 1d9 qk ey 9f9 chunks vc a7 ax 

tag= 2 mp vy s6 student e1 n6 mf0 runner iq 030 

tag= 3 processing stealth joke 2i 6f cd posts 0_ ether bz 

tag= 4 mw t5 6c ec 5e qq e1 dv f9d dy 

tag= 5 ei f0 devils tigers 70 calstate kt wr 6j l2 

tag= 6 io 5f a865 m9 1fpl 4t tnx 11th hitters giz 

tag= 7 1t z4 k8 13s m6 movement ei0l kp m0 jays 

tag= 8 m6 dy gk gt 0l ck 0t g8 1u glover 

tag= 9 uy gy pd bu terrorists pnei gt watchman starter hartford 

tag= 10 replies drinking aviv netnews 1z6e1t 9d kn processor cal contacts 

tag= 11 like just don know think people does time use good 

tag= 12 cz uj marlins 0t otc au 2di dial ladies mdi 

tag= 13 5g bu seperately gu fu ci ht pg nq a86 

tag= 14 w8 tiger 6v 2f albicans mm b8 gk cz 6g 

tag= 15 3h ml 6l i5 jg steel shrink cu r6 9e 

tag= 16 e0 wa m1 giz gx 7t fictional gn khz 5u 

tag= 17 hm g9p f0 m2 4b 0i qv qr_ dig pl 

tag= 18 rb tom mv md t9 wa rs232 pointer c4 tx 

tag= 19 

In [8]:
top_words = np.argsort(n_kw, axis=1)[:, -10:]

for topic in range(20):
    doc = np.zeros((1, X_train.shape[1]))
    for word in top_words[topic]:
        doc[0, word] = 1
    print('Topic {}:\t{}'.format(topic, '\t'.join(vectorizer.inverse_transform(doc)[0])))

Topic 0:	8v	bxn	dk	hm	immaculate	ns	rusty	sells	wing	xi
Topic 1:	1d9	3t	9f9	a7	ax	chunks	ey	qk	vc	wt
Topic 2:	030	e1	iq	mf0	mp	n6	runner	s6	student	vy
Topic 3:	0_	2i	6f	bz	cd	ether	joke	posts	processing	stealth
Topic 4:	5e	6c	dv	dy	e1	ec	f9d	mw	qq	t5
Topic 5:	6j	70	calstate	devils	ei	f0	kt	l2	tigers	wr
Topic 6:	11th	1fpl	4t	5f	a865	giz	hitters	io	m9	tnx
Topic 7:	13s	1t	ei0l	jays	k8	kp	m0	m6	movement	z4
Topic 8:	0l	0t	1u	ck	dy	g8	gk	glover	gt	m6
Topic 9:	bu	gt	gy	hartford	pd	pnei	starter	terrorists	uy	watchman
Topic 10:	1z6e1t	9d	aviv	cal	contacts	drinking	kn	netnews	processor	replies
Topic 11:	does	don	good	just	know	like	people	think	time	use
Topic 12:	0t	2di	au	cz	dial	ladies	marlins	mdi	otc	uj
Topic 13:	5g	a86	bu	ci	fu	gu	ht	nq	pg	seperately
Topic 14:	2f	6g	6v	albicans	b8	cz	gk	mm	tiger	w8
Topic 15:	3h	6l	9e	cu	i5	jg	ml	r6	shrink	steel
Topic 16:	5u	7t	e0	fictional	giz	gn	gx	khz	m1	wa
Topic 17:	0i	4b	dig	f0	g9p	hm	m2	pl	qr_	qv
Topic 18:	c4	md	mv	pointer	rb	rs232	t9	tom	tx	wa
Topic 19